In [ ]:
#!pip install --no-cache-dir -I pillow
!pip install hdf5storage
#!pip install http://download.pytorch.org/whl/cu92/torch-1.6.0-cp36-cp36m-linux_x86_64.whl
#!pip install torch
!pip3 install torchvision
#!git clone https://github.com/lanpa/tensorboardX && cd tensorboardX && python setup.py install
!pip install tensorboardX
!pip install --pre torch -f  https://download.pytorch.org/whl/nightly/cu101/torch-1.7.0.dev20200626%2Bcu101-cp36-cp36m-linux_x86_64.whl


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/")
path=os.getcwd() 

In [23]:
### 
from __future__ import division
from scipy import interpolate
import random
import os
import os.path
import h5py
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import PIL

import scipy.io
import matplotlib
matplotlib.use('Agg')
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

import torchvision.utils as utils
from tensorboardX import SummaryWriter
###
import torch
import torch.nn as nn
import argparse
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.utils.data as udata
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.utils as utils
import time
import scipy.io as sio
import logging
import hdf5storage
import datetime
from math import sqrt
%matplotlib inline
import scipy.io as spio
from scipy.interpolate import PchipInterpolator
from bisect import bisect
from google.colab import output
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pylab import cm

###

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, evaluation):
    """Save the checkpoint."""
    state = {
            'epoch': epoch,
            'iter': iteration,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }
    torch.save(state, os.path.join(model_path, 'MSIsRecon_{}.pkl'.format(evaluation)))

def plot_spectrum_msi(real, fake, epoch, i):
    x =np.linspace(400, 900, 5, endpoint=True) # the wavebands of the hyperspectral image
    fig = Figure()
    canvas = FigureCanvasAgg(fig)
    ax = fig.gca()
    #ax.set_ylim(0, 1)
    plot_real,  = ax.plot(x, real, 'ko-')
    plot_fake,  = ax.plot(x, fake, 'r.-')
    fig.legend((plot_real,plot_fake), ('real', 'fake'))
    canvas.draw()
    fig.savefig(os.path.join(iteration_path, "{}_test_{}.png".format(epoch,i)))
    I = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    I = I.reshape(canvas.get_width_height()[::-1]+(3,))
    I = np.transpose(I, [2,0,1])
    return np.float32(I)


def plot_spectrum_3(real, x_real, fake, epoch, i):
    x =np.linspace(400, 900, 3, endpoint=True) # the wavebands of the hyperspectral image
    fig = Figure()
    canvas = FigureCanvasAgg(fig)
    ax = fig.gca()
    #ax.set_ylim(0, 1)
    plot_real,  = ax.plot(x, real, 'ko-')
    plot_x_real,  = ax.plot(x, x_real, 'g.-')
    plot_fake,  = ax.plot(x, fake, 'r.-')
    fig.legend((plot_real,plot_x_real,plot_fake), ('target', "original", 'fake'))
    canvas.draw()
    fig.savefig(os.path.join(iteration_path, "{}_test_{}.png".format(epoch,i)))
    I = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    I = I.reshape(canvas.get_width_height()[::-1]+(3,))
    I = np.transpose(I, [2,0,1])
    return np.float32(I)

def plotwithcolorbar(img, title=None, figsize=(10,10)):
    ''' Plot an image with a colorbar '''
    vmin = np.min(img)
    vmax = np.max(img)
    hh = img.shape[0]
    hw = img.shape[1]
    fig, axis = plt.subplots(1, 1, figsize=figsize)
    rad2 = axis.imshow(img, vmin=vmin, vmax=vmax)
    axis.set_title(title)
    divider = make_axes_locatable(axis)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    fig.colorbar(rad2, cax=cax)
    circle_b = plt.Circle((int(hw/2), int(hh/2)), 5, color='b', fill=False)
    axis.add_artist(circle_b)
    plt.close(fig)
    return fig, axis

def initialize_logger(file_dir):
    """Print the results in the log file."""
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=file_dir, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(message)s',"%Y-%m-%d %H:%M:%S")
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.INFO)
    return logger


def mrae_loss(im_true, im_fake):
    error = torch.abs(im_fake-im_true)/im_true
    rrmse = torch.mean(error.reshape(-1))
    return rrmse

def sid_loss(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    denom1 = torch.sqrt( torch.sum( torch.pow(im_true,2), dim=1))
    denom2 = torch.sqrt( torch.sum( torch.pow(im_fake,2), dim=1))
    #
    unit_t = torch.div(im_true, denom1.unsqueeze(1))
    uint_f = torch.div(im_fake, denom2.unsqueeze(1))
    #
    sid = ((unit_t - uint_f)* (unit_t.log() - uint_f.log())).sum() / (N*H*W)
    return sid

######
path=os.getcwd() 

def normalize(data):
    data_nl = data/np.amax(data)
    return data_nl
##
def gen_random_scale(img, base=4.5, rnd=3):
    np.random.seed(rnd)
    scale = base**(np.random.rand(1,1, img.shape[2],img.shape[3])-0.5)*2    
    return img*scale
##
def gen_random_scale_n(img, rnd=3):
    np.random.seed(rnd)
    scale = np.random.uniform(0.1, 1.91, (1,1, img.shape[2],img.shape[3]))   
    return img*scale
##
def data_process_list_n(path=path):
    NO_ = 1
    hyper_f = os.path.join(path,'Maize2018_ortho_msi_cropped_04052021')
    sub_h5_fd = next(os.walk(hyper_f))[1]
    sub_h5_fd.sort()
    #
    #rgb_f = hyper_f.replace("h5", "rgb")
    rgb_f = os.path.join(path,'Maize2018_ortho_rgb_cropped_04052021')
    sub_rgb_fd = next(os.walk(rgb_f))[1]
    sub_rgb_fd.sort()

    for sf in range(len(sub_h5_fd)):
        id_n = sub_h5_fd[sf].split("_")[2][4:]
        #
        filenames_hyper = glob.glob(os.path.join(hyper_f,sub_h5_fd[sf],'*.h5'))
        filenames_rgb = glob.glob(os.path.join(rgb_f, sub_rgb_fd[sf].replace("h5", "rgb"),'*.png'))
        filenames_hyper.sort()
        print(filenames_hyper)
        filenames_rgb.sort()
        print(filenames_rgb)
        h5f = h5py.File('train_040521_maize_NatColor_final_{}.h5'.format(id_n), 'w')  ## the file concatenated hsi and rgb in one file
        
        for i in range(len(filenames_hyper)):
            print("\n")
            print(filenames_hyper[i], filenames_rgb[i])
            # load hyperspectral image
            mat =  h5py.File(filenames_hyper[i],'r')
            hyper = np.float32(np.array(mat['img']))
            print(("hyper_shape",hyper.shape))
            hyper = np.transpose(hyper, [2,0,1])#/32768.0 # because it 32768.0 was divided already before cropping into 512x512 squares

            print("hyper_test_max", np.amax(hyper))
            print("hyper_test_min",np.amin(hyper))
            mat.close()
            # load rgb image
            rgb =  cv2.imread(filenames_rgb[i])
            rgb=cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)

            rgb = np.float32(np.transpose(rgb, [2,0,1])/255.0) # change to 0-1 range

            print("rgb_test_max", np.amax(np.float32(rgb)))
            print("rgb_test_min", np.amin(np.float32(rgb)))

            data = np.concatenate((hyper,rgb), 0)
            h5f.create_dataset(str(NO_), data=data)
            NO_ += 1
        h5f.close()
        print("NO. of samples: {}".format(NO_-1))
##
class HyperDataset_list_maize_RGB(udata.Dataset):
    def __init__(self, crop_size=64):
        self.crop_size = crop_size
        xx = ["train_040521_maize_NatColor_final_1030.h5","train_040521_maize_NatColor_final_1109.h5", "train_040521_maize_NatColor_final_1119.h5","train_040521_maize_NatColor_final_1220.h5"] #"train_uavf_maize_NatColor_1109.h5" was removed because of over exposure
        xx.sort()
        # print(xx)
        key_is = []
        for i in range(len(xx)):
            f_i = xx[i]
            h5f_i = h5py.File(f_i, 'r')
            keys_i = list(h5f_i.keys())
            keys_i.sort()
            key_i_n  = ["_".join([str(i), x]) for x in keys_i]
            key_is = key_is + key_i_n
            #print(("key_is", key_is))
            h5f_i.close()
        
        self.keys = key_is
        
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        xx = ["train_040521_maize_NatColor_final_1030.h5","train_040521_maize_NatColor_final_1109.h5", "train_040521_maize_NatColor_final_1119.h5","train_040521_maize_NatColor_final_1220.h5"] #"train_uavf_maize_NatColor_1109.h5" was removed because of over exposure
        xx.sort()
        key = str(self.keys[index])
        key_path_i = key.split("_")[0]
        key_key = key.split("_")[1]
        h5f_a = h5py.File(xx[int(key_path_i)], 'r')
        #
        data_a = np.array(h5f_a[key_key])
        data = torch.as_tensor(data_a)
        # crop
        w = int(data.size()[1])
        h = int(data.size()[2])
        th, tw = self.crop_size, self.crop_size
        if w > tw or h > th:
            i = 0
            j = 0
            data = data[:,i:i+th,j:j+tw]
        h5f_a.close()
        return data[0:5,:,:], data[5:8,:,:]

class HyperDataset_list_rice_RGB(udata.Dataset):
    def __init__(self, crop_size=64):
        self.crop_size = crop_size
        xx = ["train_uavf_rice_NatColor_final_0821.h5","train_uavf_rice_NatColor_final_1026.h5","train_uavf_rice_NatColor_final_1124.h5"] #"train_uavf_rice_NatColor_final_0927.h5" was removed due to over exposure
        xx.sort()
        # print(xx)
        key_is = []
        for i in range(len(xx)):
            f_i = xx[i]
            h5f_i = h5py.File(f_i, 'r')
            keys_i = list(h5f_i.keys())
            keys_i.sort()
            key_i_n  = ["_".join([str(i), x]) for x in keys_i]
            key_is = key_is + key_i_n
            h5f_i.close()
        
        self.keys = key_is
        
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        xx = ["train_uavf_rice_NatColor_final_0821.h5","train_uavf_rice_NatColor_final_1026.h5","train_uavf_rice_NatColor_final_1124.h5"] #"train_uavf_rice_NatColor_final_0927.h5" was removed due to over exposure
        xx.sort()
        key = str(self.keys[index])
        key_path_i = key.split("_")[0]
        key_key = key.split("_")[1]
        h5f_a = h5py.File(xx[int(key_path_i)], 'r')
        #
        data_a = np.array(h5f_a[key_key])
        data = torch.as_tensor(data_a)
        # crop
        w = int(data.size()[1])
        h = int(data.size()[2])
        th, tw = self.crop_size, self.crop_size
        if w > tw or h > th:
            i = 0
            j = 0
            data = data[:,i:i+th,j:j+tw]
        h5f_a.close()
        return data[0:5,:,:], data[5:8,:,:]

def batch_MRAE(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).reshape(N, C*H*W)
    mse = nn.MSELoss(reduction='none')
    err = mse(Itrue, Ifake).sqrt_().div_(Itrue).sum(dim=1, keepdim=True).div_(C*H*W)
    return torch.mean(err)

def batch_RMSE(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).reshape(N, C*H*W)
    mse = nn.MSELoss(reduction='none')
    err = mse(Itrue, Ifake).sum(dim=1, keepdim=True).div_(C*H*W).sqrt_()
    return torch.mean(err)

def batch_SID(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clone().reshape(N, C, H*W)
    Ifake = im_fake.clone().reshape(N, C, H*W)
    denom1 = torch.pow(Itrue,2).sum(dim=1).sqrt_().reshape(N, H*W)
    denom2 = torch.pow(Ifake,2).sum(dim=1).sqrt_().reshape(N, H*W)
    #
    unit_t = torch.div(Itrue, denom1.unsqueeze(1))
    uint_f = torch.div(Ifake, denom2.unsqueeze(1))
    #
    sid = ((unit_t - uint_f)* (unit_t.log() - uint_f.log())).sum(dim = 1).reshape(N, H*W)
    sid = sid.sum() / (N*H*W)
    return sid

def weights_init_kaimingNormal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)
    elif classname.find('InstanceNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)

## find the "keys"
h5f = h5py.File('train_uavf_maize_natlk_2_1220.h5', 'r')
keys = list(h5f.keys())
keys.sort()
h5f.close()

In [ ]:
## process the data
# data_process_list_n(path=path) ## generate "train_uavf_rice_noco_2_1124.h5"


In [ ]:
######   MSI reconstruction model

class residual_block(nn.Module):
    def __init__(self):
        super(residual_block, self).__init__()
        self.conv1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = torch.add(out,residual) 
        out = self.relu2(out)
        return out
    
#####             Reconstruction_model
    
class Reconstruction_model(nn.Module):
    def __init__(self, block, block_num, input_channel, output_channel):
        super(Reconstruction_model, self).__init__()

        self.in_channels = input_channel
        self.out_channels = output_channel
        self.input_conv = nn.Conv2d(self.in_channels, 64, kernel_size=3, stride=1, padding=1, bias=True) 
        self.seq_res_block = self.stack_resBlock(block, block_num)
        self.conv = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.output_conv = nn.Conv2d(64, self.out_channels, kernel_size=3, stride=1, padding=1, bias=True)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n=m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0,sqrt(2./n))# the devide  2./n  carefully  
    def stack_resBlock(self,block,num_blocks):
        blocks = []
        for i in range(num_blocks):
            blocks.append(block()) # there is a () 
        return nn.Sequential(*blocks)   
    def forward(self, x):
        out = self.input_conv(x)
        residual = out
        out = self.seq_res_block(out)
        out = self.conv(out)
        out = torch.add(out,residual)  
        out = self.relu(out)
        out = self.output_conv(out)
        return out


In [25]:

# Training 
def train(train_data_loader, model, criterion_1, criterion_2, criterion_3, optimizer, iteration, init_lr ,epoch, w1, w2, w3):

    losses = AverageMeter()
    average_MRAE = 0.
    average_RMSE = 0. 
    average_SAM = 0.
    average_SID =0.
    
    num=len(train_data_loader)
    print('num.{}'.format(num))
    #optimizer.zero_grad()
    
    for i, (labels_gt, images_gt) in enumerate(train_data_loader):
    #for i in range(num):
        # Normalize the RGB vectors
        images_gt = torch.as_tensor(gen_random_scale_n(images_gt)).float().cuda(non_blocking=True) #+ torch.as_tensor(1e-8).float().cuda(async=True)
        images_gt = images_gt.clamp(1e-8, 1.0 - 1e-8)
        # put it on GPU
        labels_gt = torch.as_tensor(gen_random_scale_n(labels_gt)).float().cuda(non_blocking=True) #+ torch.as_tensor(1e-8).float().cuda(async=True)
        labels_gt = labels_gt.clamp(1e-8, 1.0 - 1e-8)
        #
        images = images_gt.cuda(non_blocking=True)
        #
        labels = labels_gt.cuda(non_blocking=True)
        #
        images = Variable(images, requires_grad=True)
        labels = Variable(labels, requires_grad=True)    
        
        # Decaying Learning Rate
        lr = init_lr 
        iteration = iteration + 1
        #
        optimizer.zero_grad()
        hs_scaled = model.forward(images)
        hs_scaled[hs_scaled.isnan()] = 1e-8
        #
        hs_scaled = hs_scaled.clamp(1e-8, float("Inf"))
        # remove extremes
        hs_scaled[labels==1e-8] = 1e-8
        #
        loss_1 = criterion_1(labels, hs_scaled)
        loss_2 = criterion_2(labels, hs_scaled)
        loss_3 = criterion_3(labels, hs_scaled)
        # control the contribution of each criterion through the coefficients 
        loss = loss_1*w1 + loss_2*w2 + loss_3*w3

        loss.backward()
        optimizer.step()
        #####
        MRAE = batch_MRAE(labels, hs_scaled)
        RMSE = batch_RMSE(labels, hs_scaled)
        SID = batch_SID(labels, hs_scaled)
        #
        average_MRAE    += MRAE.item()
        average_RMSE    += RMSE.item()
        average_SID    += SID.item()
        #
        losses.update(loss.item())
    return losses.avg, average_MRAE/num, average_RMSE/num, average_SID/num, iteration, lr

# Validation

def validate(val_loader, model, criterion_1, criterion_2, criterion_3, epoch, w1, w2, w3):
    
    model.eval()
    losses = AverageMeter()
    xxx=0
    num = len(val_loader)
    
    average_MRAE    =0.
    average_RMSE    =0.
    average_SAM    =0.
    average_SID    =0.
    
    for i, (target_gt, input_gt) in enumerate(val_loader):
        # Normalize the RGB vectors
        input_gt = torch.as_tensor(gen_random_scale_n(input_gt)).float().cuda(non_blocking=True) #+ torch.as_tensor(1e-8).float().cuda(async=True)
        input_gt = input_gt.clamp(1e-8, 1.0 - 1e-8)

        input = input_gt.cuda(non_blocking=True)
        target = torch.as_tensor(gen_random_scale_n(target_gt)).float().cuda(non_blocking=True) #+ torch.as_tensor(1e-8).float().cuda(async=True)
        target = target.clamp(1e-8, 1.0 - 1e-8)
        #
        target = target.cuda(non_blocking=True)
        #
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
    
        # compute hs
        with torch.no_grad():
            hs_scaled = model.forward(input_var)
        hs_scaled[hs_scaled.isnan()] = 1e-8
        hs_scaled = hs_scaled.clamp(1e-8, float("Inf"))
        # remove extremes
        hs_scaled[target_var==1e-8] = 1e-8
        #
        #
        loss_1 = criterion_1(target_var, hs_scaled)
        loss_2 = criterion_2(target_var, hs_scaled)
        loss_3 = criterion_3(target_var, hs_scaled)
        # control the contribution of each criterion through the coefficients 
        loss = loss_1*w1 + loss_2*w2 + loss_3*w3
        #####
        MRAE = batch_MRAE(target_var, hs_scaled)
        RMSE = batch_RMSE(target_var, hs_scaled)
        SID = batch_SID(target_var, hs_scaled)
        #
        average_MRAE    += MRAE.item()
        average_RMSE    += RMSE.item()
        average_SID    += SID.item()
        #####
        ## generate a figure compare the reconstructed spectra and ground truth, every epoch
        if epoch%1==0:
            #print(i)
            if (i+1)%10==0:
                xxx += 1
                H = target_var.size()[2]
                W = target_var.size()[3]
                real_spectrum = target_var.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                fake_spectrum = hs_scaled.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                I_spectrum = plot_spectrum_msi(real_spectrum, fake_spectrum, "val_"+str(epoch),i)
                ##
                input_gt_plot = input_gt[0,:,:,:].squeeze().permute(1,2,0).data.cpu().numpy()
                fig, axis = plotwithcolorbar(input_gt_plot, "fake image" )
                fig.savefig(os.path.join(iteration_path, "{}_test_fake_img_{}.png".format(epoch,i)))
                ##
                print("val_num: " + str(xxx))
        #  record loss
        losses.update(loss.item())
    return losses.avg, average_MRAE/num, average_RMSE/num, average_SID/num



In [ ]:
###               spectral reconstruction model 

# the coefficients, whether 0 or >0 to decide whether single or composite loss functions to be used
# the coefficients determine the contribution of each loss function to the final loss
w1 = 0 # mrae_loss
w2 = 1 # mse_loss
w3 = 0 # sid_loss
# number of residual blocks, should be tuned to find the optimal number
num_resBlock = 3
# decide which loss function to use
criterion_1 = mrae_loss
criterion_2 = nn.MSELoss()
criterion_3 = sid_loss 
#
# 
epoch_total = 10000
init_lr = 1e-3
#
cudnn.benchmark = True
## input and output image channels
rgb_channels = 3
msi_channels = 5
## load dataset
print("\nloading dataset ...\n")
#
trainDataset = HyperDataset_list_maize_RGB(crop_size=600)  ## here not the training data but the whole data set for this work

# set the ratio of training and validation set
validation_split = (1/3)

dataset_len = len(trainDataset) #trainDataset
indices = list(range(dataset_len))

# Randomly splitting indices:
val_len = int(np.ceil(validation_split * dataset_len))
validation_idx = np.random.choice(indices, size=val_len, replace=False)
train_idx = list(set(indices) - set(validation_idx))

## Defining the samplers for each phase based on the random indices:
train_sampler = SubsetRandomSampler(train_idx)
validation_sampler = SubsetRandomSampler(validation_idx)

# Data Loader (Input Pipeline)
train_data_loader = DataLoader(dataset=trainDataset,
                                sampler=train_sampler,
                                num_workers=1,  
                                batch_size=1,
                                shuffle=False,
                                pin_memory=True)

val_loader = DataLoader(dataset=trainDataset,
                        sampler=validation_sampler,
                        num_workers=1, 
                        batch_size=1,
                        shuffle=False,
                        pin_memory=True)
# model assembly               
model = Reconstruction_model(residual_block, num_resBlock, rgb_channels, msi_channels)
# apply kaiming normal initialization
model.apply(weights_init_kaimingNormal)
# check whether it is possible to do multi-GPU processing
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
if torch.cuda.is_available():
    model.cuda()

iteration = 0

## set the original values for the evaluation
#
record_ts_loss= np.inf
record_ts_MRAE = np.inf
record_ts_RMSE = np.inf
record_ts_SID = np.inf 
#
iteration_folder = "UAV_Maize_specReconsruction_w1-{}_w2-{}_w3-{}".format(w1, w2, w3)
model_folder = "UAV_Maize_specReconsruction_w1-{}_w2-{}_w3-{}".format(w1, w2, w3)
#
iteration_path = os.path.join(os.getcwd(), iteration_folder)
if not os.path.exists(iteration_path):
    os.makedirs(iteration_path)

optimizer=torch.optim.Adamax(model.parameters(), lr=init_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)

model_path = os.path.join(os.getcwd(), model_folder)
if not os.path.exists(model_path):
    os.makedirs(model_path)
loss_csv = open(os.path.join(model_path, 'w123_{}_{}_{}_loss.csv'.format(w1,w2,w3)), 'w+')

log_dir = os.path.join(model_path,'w123_{}_{}_{}_train.log'.format(w1,w2,w3))
logger = initialize_logger(log_dir)

# load the trained model if there is any or put "" instead
resume_file = "" #os.path.join(model_path, "MSIs_reconstruction_trained.pkl") #"" # can be specified if there is already trained model 
if resume_file:
    print("=> loading checkpoint '{}'".format(resume_file))
    checkpoint = torch.load(resume_file)
    start_epoch = checkpoint['epoch']
    iteration = checkpoint['iter']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
for epoch in range(epoch_total): ## specify the number of epochs to run
    start_time = time.time()         
    tr_loss, tr_MRAE, tr_RMSE, tr_SID, iteration, lr = train(train_data_loader, model, criterion_1, criterion_2, criterion_3, optimizer, iteration, init_lr, epoch, w1,w2,w3)
    #
    # here to implement constant learning rate
    end_time = time.time() # only record the training time
    epoch_time = end_time - start_time
    print(datetime.datetime.now())
    #lr=init_lr
    ts_loss, ts_MRAE, ts_RMSE, ts_SID = validate(val_loader, model, criterion_1, criterion_2, criterion_3, epoch, w1,w2,w3)
    #
    # Save model
    ########################### on maize validation
    # (1) ts_loss evaluation
    print("old_ts_loss", record_ts_loss, "old_ts_MRAE", record_ts_MRAE, "old_ts_RMSE",record_ts_RMSE, "old_ts_SID",record_ts_SID)
    # because weight is not added to validation loss, will save any model with lower loss rather than any individual loss
    if ts_loss < record_ts_loss: #and ts_MRAE < record_ts_MRAE or ts_loss < record_ts_loss and ts_SID < record_ts_SID 
        record_ts_loss = ts_loss
        save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_loss")
        record_ts_MRAE = ts_MRAE
        record_ts_RMSE = ts_RMSE
        record_ts_SID = ts_SID

        print("ts_loss")
        print("updated_ts_loss", record_ts_loss,"updated_ts_MRAE", record_ts_MRAE, "updated_ts_RMSE",record_ts_RMSE, "updated_ts_SID",record_ts_SID)
    
    # print loss 
    print ("Epoch:{}, Iter:{}, Time:{}, learning rate:{}, Train loss:{}, Train MRAE:{}, Train RMSE:{}, Train SID:{}".format(
        epoch, iteration, epoch_time, lr, tr_loss, tr_MRAE, tr_RMSE, tr_SID))
    # 
    logger.info("Epoch{}, Iter{}, Time:{}, learning rate:{}, Train loss:{},Train MRAE:{},Train RMSE:{}, Train SID:{}, Test loss:{},Test MRAE:{},Test RMSE:{}, Test SID:{}".format(
        epoch, iteration, epoch_time, lr, tr_loss, tr_MRAE,tr_RMSE, tr_SID, ts_loss, ts_MRAE,ts_RMSE, ts_SID))

    ###
    print("w1: {}, w2: {}, w3: {}".format(w1,w2,w3))
    if (epoch+2)%5 ==0:
        init_lr = init_lr*0.8
        print(("updated learning rate", init_lr))